In [50]:
from selenium import webdriver
# import By selectors
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib import parse
from time import sleep
import requests
import json

In [26]:
# use selenium to scrape duckduckgo results for the search term "powered by city hive"
results = set()
driver = webdriver.Chrome()
term = "\"powered by city hive\""
url = "https://html.duckduckgo.com/html/"

# type the search term into the search box
driver.get(url)
# wait for the page to load
sleep(2)

# initial search
search_box = driver.find_element(By.NAME, "q")
print(search_box)
search_box.send_keys(term)
driver.find_element(By.NAME, "b").click()

going = True
while going:
    curr_results = driver.find_elements(By.CLASS_NAME, "result__a")
    for result in curr_results:
        results.add(result.get_attribute("href"))
    try:
        # find the button with the value "Next"
        next_button = driver.find_element(By.XPATH, "//input[@value='Next']")
        # click the button
        next_button.click()

        sleep(2)
    except:
        going = False
        print("done")


<selenium.webdriver.remote.webelement.WebElement (session="2f65c5a742e4917106af4142c226efa7", element="72D468F64608EF1442F218ADEDB4F645_element_2")>
done


In [62]:
# A utility function that returns all of the values associated with given keys from a dictionary
# def get_all_values(key, dictionary):
#     values = []
#     for k, v in dictionary.items():
#         if k == key:
#             values.append(v)
#         elif isinstance(v, dict):
#             results = get_all_values(key, v)
#             for item in results:
#                 values.append(item)
#         elif isinstance(v, list):
#             for d in v:
#                 if isinstance(d, dict):
#                     more_results = get_all_values(key, d)
#                     for item in more_results:
#                         values.append(item)
#     return values
def get_all_values(keys, dictionary):
    values = {key: [] for key in keys}
    for k, v in dictionary.items():
        if k in keys:
            values[k].append(v)
        elif isinstance(v, dict):
            results = get_all_values(keys, v)
            for key in keys:
                values[key].extend(results[key])
        elif isinstance(v, list):
            for d in v:
                if isinstance(d, dict):
                    more_results = get_all_values(keys, d)
                    for key in keys:
                        values[key].extend(more_results[key])
    return values

In [65]:
def get_cityhive_info(url, driver: webdriver.Chrome):
    # gets info about a cityhive site from its url
    # returns a dictionary with the following keys (to match the provider schema)
    # name: string;
    # location: [number, number];
    # description: string;

    # base_url: string;
    # storefront_page: string;
    driver.get(url)
    info = {
        "name": None,
        "location": None,
        "description": None,
        "base_url": url,
        "storefront_page": None
    }
    # find the ch-elements.search-box element
    search_box = driver.find_element(By.TAG_NAME, "ch-elements.search-box")
    # get the shop-page attribute
    info['storefront_page'] = search_box.get_attribute("shop-page")

    try:
        ch_cont = driver.find_element(By.TAG_NAME, "ch-elements.container")
        ch_cont_content = json.loads(ch_cont.get_attribute("container"))
        dat = get_all_values(["merchant_name", "coordinates", "full_address"], ch_cont_content)
        info['name'] = dat['merchant_name'][0]
        info['location'] = dat['coordinates'][0]
        info['description'] = dat['full_address'][0]
    except:
        driver.get(f"{info['storefront_page']}")
        ch_cont = driver.find_element(By.TAG_NAME, "ch-elements.container")
        ch_cont_content = json.loads(ch_cont.get_attribute("container"))
        dat = get_all_values(["merchant_name", "coordinates", "full_address"], ch_cont_content)
        info['name'] = dat['merchant_name'][0]
        info['location'] = dat['coordinates'][0]
        info['description'] = dat['full_address'][0]

    return info

In [68]:
driver = webdriver.Chrome()
info = get_cityhive_info("https://spiritsunlimited.com/", driver)
print(info)

{'name': 'Spirits Unlimited 89', 'location': [-74.154467, 39.913577], 'description': '437 Atlantic City Blvd, Bayville, NJ 08721, USA', 'base_url': 'https://spiritsunlimited.com/', 'storefront_page': 'https://spiritsunlimited.com/shop/'}


In [73]:
with open('ddg_results.txt', 'r') as f:
    results = f.readlines()
    results = [result.strip() for result in results]
    # parse each line as a url, and only keep the base url and protocol
    results = [parse.urlparse(result).scheme + "://" + parse.urlparse(result).netloc for result in results]
    results = set(results)

shops = []
driver = webdriver.Chrome()
for result in results:
    input("press enter to continue")
    try:
        shops.append(get_cityhive_info(result, driver))
        print("got info for " + result)
    except Exception as e:
        print("failed to get info for " + result)
        print(e)
        


failed to get info for https://twinliquors.com
Message: no such element: Unable to locate element: {"method":"tag name","selector":"ch-elements.container"}
  (Session info: chrome=114.0.5735.133); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000101103f48 chromedriver + 4226888
1   chromedriver                        0x00000001010fc4f4 chromedriver + 4195572
2   chromedriver                        0x0000000100d40d68 chromedriver + 281960
3   chromedriver                        0x0000000100d7bfc8 chromedriver + 524232
4   chromedriver                        0x0000000100db2c58 chromedriver + 748632
5   chromedriver                        0x0000000100d6ff1c chromedriver + 474908
6   chromedriver                        0x0000000100d70ef4 chromedriver + 478964
7   chromedriver                        0x00000001010c559c chromedriver + 397

KeyboardInterrupt: Interrupted by user